In [ ]:
from google_api import get_form
from data_processing import clean_sheet

# Get the form responses
get_form()
clean_sheet()

In [ ]:
import json
from aws import *

key = 'records.json'
obj = download_object(bucket_name, key)
records = json.loads(obj)

records

In [ ]:
from data_processing import *
import json

records = write_new_records_to_json(None)

json.dumps(records)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time

def test_duckduckgo_search():
    # Set up Chrome options for headless mode
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Ensure GUI is not displayed
    chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
    chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

    # Initialize WebDriver with options
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # Navigate to DuckDuckGo
        driver.get("https://www.duckduckgo.com")
        assert "DuckDuckGo" in driver.title

        # Find the search box using its name attribute value
        search_box = driver.find_element("name", "q")
        search_box.clear()

        # Perform a search operation
        search_term = "selenium headless browser testing"
        search_box.send_keys(search_term)
        search_box.send_keys(Keys.RETURN)  # Simulate pressing the Enter key

        time.sleep(2)  # Wait a bit for the search results to load

        # Verify that the search operation was successful
        assert search_term in driver.title

        print("Test passed: Search operation was successful.")
    except AssertionError as e:
        print(f"Test failed: {e}")
    finally:
        driver.quit()

test_duckduckgo_search()


In [2]:
from scripts.data_processing import *
from scripts.google_api import *

checkin_date = '29-03-2024'
# Get form responses
# sheet_path = get_form()
clean_df = clean_sheet()
filtered_df = filter_df_on_checkin_date(clean_df, checkin_date)

filtered_df

,Timestamp_1,Column_No,Check-in date,Check-out date,City of birth,City of residence,Country of issue,Country of residence,Date of birth,First name,Last name,Nationality,Passport (or ID) number
28,3/28/2024 7:09:06,2,29-03-2024,01-04-2024,Pelhřimov,Praha,Portugal,Czech Republic,16-04-1985,Olga,Kučerová,Czech,47515859
29,3/28/2024 7:09:06,3,29-03-2024,01-04-2024,Rakovník,Praha,Portugal,Czech Republic,04-08-1975,Tomáš,Baldýnský,Czech,207867301


In [15]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from dotenv import load_dotenv

# Load environment variables from a .env file located in the same directory as this script
load_dotenv()
uh_input_ = os.getenv('SEF_UH')
estabelecimento_input_ = os.getenv('SEF_ESTABELECIMENTO')
chave_activacao_input_ = os.getenv('SEF_CHAVE')
pdf_nif = os.getenv('PDF_NIF')
pdf_senha = os.getenv('PDF_SENHA')

# Extract the mapping files
countries_mapping = os.path.join(os.getcwd(), 'parameters', 'countries_mapping.json')
nationalities_mapping = os.path.join(os.getcwd(), 'parameters', 'nationalities_mapping.json')

def callback(msg):
    print(msg)

def fill_in_sef_form(df, callback):
    try:
        with open(countries_mapping) as f:
            countries = json.load(f)

        with open(nationalities_mapping) as f:
            nationalities = json.load(f)

        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        web = webdriver.Chrome(options=chrome_options)

        callback("Opening the SEF website...")
        url = 'https://siba.sef.pt/s/FB.aspx?ReturnUrl=%2fs%2fbal%2fLotesEnvio.aspx'
        web.get(url)
        time.sleep(2)

        callback("Filling in the establishment details...")
        uh_input = web.find_element('id', 'Conteudo_txtUH')
        estabelecimento_input = web.find_element('id', 'Conteudo_txtEstabelecimento')
        chave_activacao_input = web.find_element('id', 'Conteudo_txtChaveActivacao')

        uh_input.send_keys(uh_input_)
        estabelecimento_input.send_keys(estabelecimento_input_)
        chave_activacao_input.send_keys(chave_activacao_input_)

        # submit_button = web.find_element('id', 'Conteudo_btnConfirmar')
        # submit_button.click()

        # Example for the submit button
        # submit_button = WebDriverWait(web, 10).until(
        #     EC.element_to_be_clickable((By.ID, 'Conteudo_btnConfirmar'))
        # )
        # submit_button.click()

        # submit_button = web.find_element('id', 'Conteudo_btnConfirmar')
        # ActionChains(web).move_to_element(submit_button).perform()
        # submit_button.click()

        submit_button = web.find_element('id', 'Conteudo_btnConfirmar')
        web.execute_script("arguments[0].click();", submit_button)

        time.sleep(2)

        callback("Creating a new list...")
        new_list = web.find_element('id', 'Conteudo_btnNovaLista')
        new_list.click()
        time.sleep(2)

        callback("Editing the list...")
        edit_list = web.find_element('id', 'Conteudo_dg_btnSelect_0')
        edit_list.click()
        time.sleep(2)

        try:
            callback("Trying to add new bulletin in case list already existed...")
            add_bulletin = web.find_element('id', 'Conteudo_btnNovaBAL')
            # add_bulletin.click()
            web.execute_script("arguments[0].click();", add_bulletin)
            callback("Added new bulleting...")
            time.sleep(2)
        except Exception as e:
            callback(f"Skipped adding new bulleting. Error: {e}")
            time.sleep(1)

        callback("Preparing to fill in guest details...")
        for idx, row in df.iterrows():
            name_ = web.find_element('id', 'Conteudo_txtNome')
            dob_ = web.find_element('id', 'Conteudo_txtDataNascimento')
            nationality_ = web.find_element('id', 'Conteudo_lstNacionalidade')
            country_of_res_ = web.find_element('id', 'Conteudo_lstPaisResidencia')
            passport_ = web.find_element('id', 'Conteudo_txtNumPassaporteBI')
            country_of_issue_ = web.find_element('id', 'Conteudo_lstPaisEmissor')
            checkin_date_ = web.find_element('id', 'Conteudo_txtDataEntrada')
            checkout_date_ = web.find_element('id', 'Conteudo_txtDataSaida')

            name = row['First name'] + ' ' + row['Last name']
            dob = row['Date of birth']
            nationality = countries[nationalities[row['Nationality']]]
            country_of_res = countries[row['Country of residence']]
            passport = row['Passport (or ID) number']
            country_of_issue = countries[row['Country of issue']]
            checkin_date = row['Check-in date']
            checkout_date = row['Check-out date']
            print("before sending keys")
            name_.send_keys(name)
            print("name done")
            dob_.send_keys(dob)
            print("dob_ done")
            nationality_.send_keys(nationality)
            # web.execute_script("arguments[0].value = arguments[1];", nationality_, nationality)
            print("nationality_ done")
            country_of_res_.send_keys(country_of_res)
            print("country_of_res_ done")
            passport_.send_keys(passport)
            print("passport_ done")
            country_of_issue_.send_keys(country_of_issue)
            print("country_of_issue_ done")
            checkin_date_.send_keys(checkin_date)
            print("checkin_date_ done")
            checkout_date_.send_keys(checkout_date)
            print("checkout done")

            # Using JavaScript to set the values
            # web.execute_script("arguments[0].value = arguments[1];", name_, name)
            # print("name done")
            # web.execute_script("arguments[0].value = arguments[1];", dob_, dob)
            # print("dob_ done")
            # web.execute_script("arguments[0].value = arguments[1];", nationality_, nationality)
            # print("nationality_ done")
            # web.execute_script("arguments[0].value = arguments[1];", country_of_res_, country_of_res)
            # print("country_of_res_ done")
            # web.execute_script("arguments[0].value = arguments[1];", passport_, passport)
            # print("passport_ done")
            # web.execute_script("arguments[0].value = arguments[1];", country_of_issue_, country_of_issue)
            # print("country_of_issue_ done")
            # web.execute_script("arguments[0].value = arguments[1];", checkin_date_, checkin_date)
            # print("checkin_date_ done")
            # web.execute_script("arguments[0].value = arguments[1];", checkout_date_, checkout_date)
            # print("checkout done")

            print("after sending keys")
            save = web.find_element('id', 'Conteudo_btnActualizarBAL')
            save.click()
            # web.execute_script("arguments[0].click();", save)

            callback(f"Details for {name} saved. Proceeding to the next guest...")
            time.sleep(3)

        callback("Finalizing and sending the list...")
        send = web.find_element('id', 'Conteudo_btnEnviarLista')
        # web.execute_script("arguments[0].click();", send)
        send.click()
        time.sleep(3)

        callback("SEF registration completed successfully.")

    except Exception as e:
        callback(f"Error during SEF registration: {e}")

    finally:
        web.quit()

fill_in_sef_form(filtered_df, callback)

Opening the SEF website...
Filling in the establishment details...
Creating a new list...
Editing the list...
Trying to add new bulletin in case list already existed...
Added new bulleting...
Preparing to fill in guest details...
before sending keys
name done
dob_ done
nationality_ done
country_of_res_ done
passport_ done
country_of_issue_ done
checkin_date_ done
checkout done
after sending keys
Details for Olga  Kučerová saved. Proceeding to the next guest...
before sending keys
name done
dob_ done
nationality_ done
country_of_res_ done
passport_ done
country_of_issue_ done
checkin_date_ done
checkout done
after sending keys
Details for Tomáš Baldýnský saved. Proceeding to the next guest...
Finalizing and sending the list...
SEF registration completed successfully.
